In [1]:
import numpy as np
import math
import os
import keras
import tensorflow as tf
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils import batch_indices
from cleverhans.attacks import DeepFool
from cleverhans.utils_keras import KerasModelWrapper

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [2]:
lr = 0.001
epochs = 10
batch_size = 128 
drop_ratio = 0.10
num_inputs = int(28 * 28 * (1-drop_ratio))
noise_eps = 0.1

In [3]:
def logistic_regression_model(input_ph,num_inputs, nb_classes=10):
    model = Sequential()
    model.add(Dense(nb_classes,input_shape=(num_inputs,)))
    model.add(Activation('softmax'))
    return model

In [22]:
class ExportMNISTGraph():
    def __init__(self,seed=0):
        np.random.seed(seed)
        # Create local graph and use it in the self.session
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        X_train, Y_train, X_test, Y_test = data_mnist()
        # Flatten image matrices
        num_train, num_rows, num_cols, num_channels = X_train.shape
        num_test, _, _, _ = X_test.shape
        X_train = X_train.reshape((num_train, num_rows * num_cols * num_channels))
        X_test = X_test.reshape((num_test, num_rows * num_cols * num_channels))
        # Randomly drop some ratio of pixels
        input_indices = np.random.choice(
            num_rows * num_cols, num_inputs, replace=False)
        self.X_train = X_train[:, input_indices]
        self.X_test = X_test[:, input_indices]
        
        self.Y_train = Y_train
        self.Y_test = Y_test
        
    
    def train(self,model_name):
        with self.graph.as_default():
            # Define placeholders
            x = tf.placeholder(tf.float32, shape=(None, num_inputs))
            y = tf.placeholder(tf.float32, shape=(None, 10))

            # Wrap log reg model for applying adversarial examples
            model = logistic_regression_model(x,num_inputs)
            tf.add_to_collection(model_name, model)

            wrap = KerasModelWrapper(model, 10) 

            # Define the objective
            logits = wrap.get_logits(x)
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=logits)

            # Define the update func
            optimizer = tf.train.AdamOptimizer(learning_rate=lr)
            train_step = optimizer.minimize(loss)
            acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(y, 1), 
                                              predictions=tf.argmax(logits,1))

            # Define adv attack
            deepfool = DeepFool(wrap, sess=self.sess)
            deepfool_params = {'eps': noise_eps,
                           'clip_min': 0.,
                           'clip_max': 1.}

            # Attack images
            adv_x = deepfool.generate(x, **deepfool_params)
            # Consider the attack to be constant
#             adv_x = tf.stop_gradient(adv_x)
            tf.add_to_collection("adv", adv_x)

            # Evaluate predictions on adv attacks
            preds = model(adv_x)
            acc_adv, acc_op_adv = tf.metrics.accuracy(labels=tf.argmax(y, 1), 
                                              predictions=tf.argmax(preds,1))

            # Initialize variables
            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())
            saver = tf.train.Saver({'advx_x':adv_x})
            rng = np.random.RandomState() # for batch sampling

            for epoch in range(epochs):
                # Compute number of batches
                num_batches = int(math.ceil(float(len(self.X_train)) / batch_size))
                assert num_batches * batch_size >= len(self.X_train)

                # Indices to shuffle training set
                index_shuf = list(range(len(self.X_train)))
                rng.shuffle(index_shuf)
                for batch in range(num_batches):

                    # Compute batch start and end indices
                    start, end = batch_indices(
                        batch, len(self.X_train), batch_size)

                    # Perform one training step
                    feed_dict = {x: self.X_train[index_shuf[start:end]],
                                 y: self.Y_train[index_shuf[start:end]]}
                    self.sess.run(train_step,feed_dict=feed_dict)

                feed_dict = {x: self.X_train,
                             y: self.Y_train}
                acc_val = self.sess.run(acc_op,feed_dict=feed_dict)
                print(f"Epoch: {epoch}, Train Acc: {acc_val:.5f}")
#             # Evaluate the model on the test set
#             feed_dict = {x: self.X_test, y: self.Y_test}
#             test_acc = self.sess.run(acc_op, feed_dict=feed_dict)
#             print(f"Test Acc: {test_acc:.5f}")

#             # Evaluate the model on the adversarial test set
#             test_acc_adv = self.sess.run(acc_op_adv, feed_dict=feed_dict)
#             print(f"Test Acc Adv: {test_acc_adv:.5f}")

            model_path = f"../models/{model_name}"
            os.makedirs(model_path, exist_ok=True)
            saver.save(self.sess, f"{model_path}/{model_name}")
            print(f"Model saved to {model_path}/{model_name}")

In [23]:
for i in range(1):
    graph = ExportMNISTGraph(i)
    graph.train(f'model{i}')

TypeError: names_to_saveables must be a dict mapping string names to Tensors/Variables. Not a variable: Tensor("PyFunc:0", dtype=float32)

In [19]:
graph.graph._collections

{('__variable_store',): [<tensorflow.python.ops.variable_scope._VariableStore at 0x1218d3ac8>],
 ('__varscope',): [<tensorflow.python.ops.variable_scope._VariableScopeStore at 0x1218c1a68>],
 'adv': [<tf.Tensor 'StopGradient:0' shape=<unknown> dtype=float32>],
 'local_variables': [<tf.Variable 'accuracy/total:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy/count:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy_1/total:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy_1/count:0' shape=() dtype=float32_ref>],
 'metric_variables': [<tf.Variable 'accuracy/total:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy/count:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy_1/total:0' shape=() dtype=float32_ref>,
  <tf.Variable 'accuracy_1/count:0' shape=() dtype=float32_ref>],
 'model2': [<keras.models.Sequential at 0x121141eb8>],
 'train_op': [<tf.Operation 'Adam' type=NoOp>],
 'trainable_variables': [<tf.Variable 'dense_1/kernel:0' shape=(705, 10) dtype=floa

In [15]:
for name in graph.graph._collections:
    print(name)
    print('%s: %s' % (name, set(map(type, coll))))

trainable_variables


NameError: name 'coll' is not defined

In [ ]:
class ImportGraph():
    """  Importing and running isolated TF graph """
    def __init__(self, loc):
        # Create local graph and use it in the session
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        with self.graph.as_default():
            # Import saved model from location 'loc' into local graph
            saver = tf.train.import_meta_graph(loc + '.meta',
                                               clear_devices=True)
            saver.restore(self.sess, loc)
            # There are TWO options how to get activation operation:
              # FROM SAVED COLLECTION:            
            self.activation = tf.get_collection('activation')[0]
              # BY NAME:
            self.activation = self.graph.get_operation_by_name('activation_opt').outputs[0]

    def run(self, data):
        """ Running the activation operation previously imported """
        # The 'x' corresponds to name of input placeholder
        return self.sess.run(self.activation, feed_dict={"x:0": data})
      
      
    ### Using the class ###
    data = 50         # random data
    model = ImportGraph('models/model_name')
    result = model.run(data)